## Quantitative Structure-Property Relationships



Quantitative Structure-Property Relationships (QSPR) and Quantitative
Structure-Activity Relationships (QSAR) use statistical models to relate a set
of predictor values to a response variable. Molecules are described using a set
of *descriptors*, and then mathematical relationships can be developed to explain
observed properties. In QSPR and QSAR, physico-chemical properties of theoretical
descriptors of chemicals are used to predict either a physical property or a
biological outcome. We will use python to import data in preparation for 
QSPR and QSAR.



## Molecular Descriptors



A molecular descriptor is &ldquo;final result of a logical and mathematical procedure,
which transforms chemical information encoded within a symbolic repre-sentation
of a molecule into a useful number or the result of some standardized
experiment&rdquo; (Todeschini, R.; Consonni, V. *Molecular descriptors for
chemoinformatics* **2009** Wiley‑VCH, Weinheim). You are already familiar with
descriptors such as molecular weight or number of heavy atoms and we have
queried PubChem for data such as XLogP. We&rsquo;ll examine just a few simple
descriptors, but thousands have been developed for applications in QSPR.



## Using rdkit and mordred to calculate descriptors



Clearly we have been using algorithms for calculating these indices. This is
time consuming for an individual, but programs can be used to complete this much
easier. We will use the rdkit and mordred python libraries to help us out.



In [1]:
import json
from rdkit import Chem                            # imports the Chem module from rdkit
from mordred import Calculator, descriptors       # imports mordred descriptor library
calc = Calculator(descriptors, ignore_3D=True)    # sets up a function reading descriptors
len(calc.descriptors) # tells us how many different types of descriptors are available in the library

ImportError: cannot import name 'product' from 'numpy' (/Users/scottreed/.conda/envs/molecular_informatics/lib/python3.12/site-packages/numpy/__init__.py)

Each of these mordred calc descriptors and the resulting code syntax can be found here
[http://mordred-descriptor.github.io/documentation/master/api/modules.html](http://mordred-descriptor.github.io/documentation/master/api/modules.html)



### Molecule characteristics



We will use a simple case of having mordred count rings of different types.



### Looping through a list of molecules



In [ ]:
smiles = ["CN(c1ncc(F)cn1)[C@H]1CCCNC1", "C=CC(=O)N1CCC(CNc2ncnc(N)c2-c2ccc(Oc3ccccc3)cc2)CC1", "C1CCCC1"] 

for smile in smiles:
    mol = Chem.MolFromSmiles(smile)
    # Initialize a Mordred calculator
    calc = Calculator()
    # add a function to claculator
    calc.register(descriptors.RingCount.RingCount())
    # execute calculator and print output
    results = calc(mol)
    print(results)


In [1]:
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.ipython_useSVG = True
params = Chem.SmilesParserParams()
params.removeHs=False

IPythonConsole.molSize = (600, 600)   # Change image size
IPythonConsole.ipython_useSVG = True  # Change output to SVG

query = Chem.MolFromSmiles(
    smiles[1],
    params)
query

NameError: name 'Chem' is not defined

Now that we have an understanding on how mordred works to get our
descriptors, let&rsquo;s add more info within the loop and create a function.

Consider using http://www.scbdd.com/chemdes/ instead of mordred


In [ ]:
from rdkit import Chem
from mordred import Calculator, descriptors

def get_ring_info(smiles):
    # Convert the SMILES code to an RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return "Invalid SMILES code."

    # Initialize a Mordred calculator
    calc = Calculator()

    # Define ring count descriptors
    ring_descriptors = {
        'total_rings': descriptors.RingCount.RingCount(),
        'aromatic_rings': descriptors.RingCount.RingCount(aromatic=True),
        'non_aromatic_rings': descriptors.RingCount.RingCount(aromatic=False),
        'hetero_rings': descriptors.RingCount.RingCount(hetero=True),
        'carbon_rings': descriptors.RingCount.RingCount(hetero=False)
    }

    # Register the descriptors in the calculator
    for descriptor in ring_descriptors.values():
        calc.register(descriptor)

    # Calculate descriptors for the given molecule
    results = calc(mol)

    # Initialize an empty dictionary to store ring information
    ring_info = {}
    
    # Populate the ring_info dictionary with the calculated results
    for name, descriptor in ring_descriptors.items():
        ring_info[name] = results[descriptor]

    return ring_info

# Example usage
smiles_code = "C1=CC2=C(C=C1)C=CC=C2"  # Example: Naphthalene
ring_info = get_ring_info(smiles_code)
print(ring_info)


## Load data to analyze
This set of smiles codes, solubility and binding data comes from a pharma company

In [ ]:
# import and view drug data from : https://github.com/molecularinformatics/Computational-ADME/tree/main
# described here: https://pubs.acs.org/doi/10.1021/acs.jcim.3c00160
import os
data_dir = os.path.join('data')
file_path = os.path.join(data_dir, 'ADME_public_set_3521.csv')
df = pd.read_csv(file_path)
df.head(4) # Shows the top X entries in the dataframe


Some data is missing. Python uses Nan (not a number) to indicate a missing value.
Before analyzing and plotting this data, let's trim out rows that have Nan for solubility. Let's 
also trim the list length to the first 500 items. We'll talk about random and scaffold based splits 
in a future class.

In [ ]:
# Select rows with 'LOG SOLUBILITY PH 6.8 (ug/mL)' not NaN
non_nan_df = df.dropna(subset=['LOG SOLUBILITY PH 6.8 (ug/mL)'])

# Calculate ring properties and add to new dataframe
ring_properties_list = []
for smiles in non_nan_df['SMILES'][0:499]:
    ring_properties = get_ring_info(smiles)
    ring_properties_list.append(ring_properties)

# Create a new dataframe with ring properties
ring_properties_df = pd.DataFrame(ring_properties_list)
# ring_properties_df.head(6)
# Combine the ring properties dataframe with the original non-NaN dataframe
final_df = pd.concat([non_nan_df.reset_index(drop=True), ring_properties_df], axis=1)

final_df_trimmed = final_df[['SMILES', 'LOG SOLUBILITY PH 6.8 (ug/mL)', 'LOG MDR1-MDCK ER (B-A/A-B)',
                                     'total_rings', 'aromatic_rings', 'non_aromatic_rings', 'hetero_rings', 'carbon_rings']]
final_df_trimmed.head()# defaults to 5

## Using descriptors to predict molecular properties




For this exercise we will be using the [pandas](https://pandas.pydata.org/) (Python Data Analysis) library to
help us read, write and manage data. We will also use matplotlib to generate
graphs.


### Graphing the data



Now we can graph the data using matplotlib.



In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.scatter(final_df_trimmed['LOG SOLUBILITY PH 6.8 (ug/mL)'],final_df_trimmed['aromatic_rings'], alpha=0.7)
plt.title('Solubility vs Aromatic Ring Count')
plt.xlabel('Aromatic Ring Count')
plt.ylabel('Log Solubility (ug/mL)')
plt.grid(True)


In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(final_df_trimmed['total_rings'], final_df_trimmed['LOG MDR1-MDCK ER (B-A/A-B)'])
plt.title('LOG MDR1-MDCK ER (B-A/A-B) vs Total Ring Count')
plt.xlabel('Total Ring Count')
plt.ylabel('Log MDR1-MDCK ER')
plt.grid(False)


exercise 1: try writing a block that plots another column vs a specific type of ring count
try creating a  scatter plot from these two columns in final_df_trimmed

In [9]:
# exercise 1:


### Adding descriptors to the dataset
Let's switch to a simpler and smaller dataset to try out creation of a SQL database



In [ ]:
#load this in 
df_bp = pd.read_csv("data/BP.csv") # read in the file into a pandas dataframe
#and look through the data
df_bp.head(5)

To explore creating a database, and creating an API start with:

pip install Flask

pip install Flask-Cors

pip install sqlalchemy

And if you don't have sqlite installed (it comes with recent python installs):

pip install db-sqlite3

more info: 
https://flask.palletsprojects.com/en/2.2.x/quickstart/ 
https://docs.sqlalchemy.org/en/14/intro.html


First we will use an engine from sqlalchemy to convert our dataframe to a SQL (structured query language) database using sqlite, the SQL language built into python. The name of the database will be 'chemical'.

In [ ]:
from sqlalchemy import create_engine, text

# Create an in-memory SQLite database
engine = create_engine('sqlite://', echo=False)

# Assuming df_bp is a pandas DataFrame, write it to the database
df_bp.to_sql('chemical', con=engine, if_exists='replace', index=False)

# Use a connection object to execute queries
with engine.connect() as connection:
    # Use the sqlalchemy text() function to allow raw SQL execution
    query = text("SELECT * FROM chemical WHERE SMILES = 'CC'")
    result = connection.execute(query)

    # Fetch and print all rows
    rows = result.fetchall()
    for row in rows:
        print(row)


Once the 'chemical' database is created it can be interrogated with CRUD (create, read, update, and delete) operations using the SQLite language sent through the engine. Here we use the engine to execute a SELECT statement. In SQL the * means "all" so we are selecting all rows of the database where the column "name" has the value "Methane". Appending fetchall() to the end returns the result to us.

In [ ]:
with engine.connect() as connection:
    query = text("SELECT 'chemical'.'name' FROM 'chemical' WHERE 'chemical'.'BP_C' is -0.1")
    result = connection.execute(query)

    # Fetch and print all rows
    rows = result.fetchall()
    for row in rows:
        print(row)
# print(row[1])


exercise 2: try writing a script that returns propane by MW from your SQL database 

In [48]:
# exercise 2


We can embed this engine in a function that returns the result as a dictionary in JSON (JavaScript Object Notation) format commonly used in http requests.

In [43]:
def get_chemicals(bp_value):
    chemicals = {}
    df = pd.read_csv("data/BP.csv")
    
    # Create SQLite engine
    engine = create_engine('sqlite://', echo=False)
    
    # Save DataFrame to SQL database
    df.to_sql('chemical', con=engine, if_exists='replace', index=False)
    
    # Define column names (in case you need to map them later)
    col_names = ['index', 'compound_number', 'name', 'BP_C', 'BP_K', 'SMILES', 'MW']
    
    try:
        # Establish a connection using context manager
        with engine.connect() as connection:
            
            # Use text() to run the SQL query
            query = text("SELECT * FROM chemical WHERE BP_C = :bp_value")
            result = connection.execute(query, {'bp_value': bp_value})
            
            # Fetch all rows matching the boiling point condition as mappings (dictionary-like rows)
            rows = result.mappings().all()  # Use .mappings() to access rows by column names
            
            if not rows:
                print("No results found")
                return json.dumps({})
            
            # Prepare the dictionary of chemicals
            for row in rows:
                # `row` is now a dictionary-like object, so you can access values by column name
                chemical = {col: row[col] for col in col_names if col in row}
                chemicals[row['name']] = chemical  # Use 'name' as the key for each chemical

    except Exception as e:
        print(f"An error occurred: {e}")
        return json.dumps({})

    # Convert dictionary to JSON for output
    chemicals_out = json.dumps(chemicals, separators=(',', ':'))
    
    return chemicals_out


Now check this function out by submitting a bp_value. It must be sent as a string because of how the database was setup. More nuanced methods of creating your SQL schema where you can use integers or floating point numbers are described here: https://levelup.gitconnected.com/full-stack-web-app-with-python-react-and-bootstrap-backend-8592baa6e4eb

In [ ]:
bp_value = -0.1
get_chemicals(bp_value)

Now we will put it all together into an app using flask and create an API. Follow the directions below carefully. Running this code inside Jupyter casues problems becasue there is no easy way to shutdown your app. This block repeats material from earlier so that it is available to this standalone app.

In [ ]:
# don't run the "app.run()" line in Jupyter. 
# Instead, copy the text of  this block and save as "first_app.py" after uncommenting the last 2 lines and 
# run it from a terminal or prompt with this command: "python3 first_app.py"
# The BP.csv file must be in the same folder

import pandas as pd
from sqlalchemy import create_engine, text
from flask import Flask, request, jsonify #added to top of file
from flask_cors import CORS #added to top of file
import json

def get_chemicals(bp_value):
    chemicals = {}
    df = pd.read_csv("../data/BP.csv")
    
    # Create SQLite engine
    engine = create_engine('sqlite://', echo=False)
    
    # Save DataFrame to SQL database
    df.to_sql('chemical', con=engine, if_exists='replace', index=False)
    
    # Define column names (in case you need to map them later)
    col_names = ['index', 'compound_number', 'name', 'BP_C', 'BP_K', 'SMILES', 'MW']
    
    try:
        # Establish a connection using context manager
        with engine.connect() as connection:
            
            # Use text() to run the SQL query
            query = text("SELECT * FROM chemical WHERE BP_C = :bp_value")
            result = connection.execute(query, {'bp_value': bp_value})
            
            # Fetch all rows matching the boiling point condition as mappings (dictionary-like rows)
            rows = result.mappings().all()  # Use .mappings() to access rows by column names
            
            if not rows:
                print("No results found")
                return json.dumps({})
            
            # Prepare the dictionary of chemicals
            for row in rows:
                # `row` is now a dictionary-like object, so you can access values by column name
                chemical = {col: row[col] for col in col_names if col in row}
                chemicals[row['name']] = chemical  # Use 'name' as the key for each chemical

    except Exception as e:
        print(f"An error occurred: {e}")
        return json.dumps({})

    # Convert dictionary to JSON for output
    chemicals_out = json.dumps(chemicals, separators=(',', ':'))
    
    return chemicals_out

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

@app.route('/api/chemical/<name>', methods=['GET','POST'])

def api_get_users(name):
    return jsonify(get_chemicals(name))

if __name__ == ('__main__'):
    app.run()

In [ ]:
import requests
mydata = '-0.1'
url = 'http://127.0.0.1:5000/api/chemical/-0.1'
req = requests.post(url, data = mydata)
print(req.text)

Assignment, option 1:

create a second SQL database from the solubility data we started with. Create a query that returns a 
smiles(or many) based on an input solubility value


Assignment, option 2:

Add a range input option.



Hints:

You can do the API assignment using a GET instead of a POST. The main issue to solve is that you need to pass a variable following the variable rules discussed here: https://flask.palletsprojects.com/en/2.2.x/quickstart/#routingLinks to an external site.

And after getting the variable from the API it needs to get passed again to get_chemicals.

It is easier to troubleshoot in Jupyter compared to running scripts from terminal/command prompt. I’d recommend doing as much work as you can in Jupyter before saving the script and moving to terminal.

Save your file ending in .py

When you run your script in terminal or prompt, make sure that you are in an environment that has flask, flask_cors, sqlalchemy, json, and pandas installed.

You can check what packages are available in a given environment by typing “pip list in your terminal. If you are unable to navigate to the environment you want, you can always create a new one with conda from the terminal and install the packages you need.

